# 2019年度　パターン認識最終課題

パターン認識最終課題では、銀行の顧客に関するデータセットを使って、顧客の婚姻状況を予測するモデルを構築してもらいます。

学籍番号と氏名を記入してください。
記入例：1811000 先端　太郎

1911311 CONGXI

## 課題１: 色々なモデルを試してみよう！
まずは、講義で扱った様々な識別器を試してみましょう。
以下に示すロジスティック回帰の例を参考に、指示に従いSVM、K-近傍法、決定木、ランダムフォレストの実装を行ってもらいます。

In [1]:
from IPython.display import display
import pandas as pd
from pandas import DataFrame, Series
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

### Step1 データの読み込み
まずは、データを読み込みます。

In [2]:
df_train_data = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [8]:
df_train_data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,58,housemaid,married,basic.4y,unknown,yes,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.970,5228.1,no
1,38,technician,married,professional.course,no,yes,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
2,81,retired,divorced,basic.4y,no,yes,no,cellular,oct,thu,...,1,999,1,failure,-1.1,94.601,-49.5,0.987,4963.6,no
3,49,blue-collar,married,basic.6y,no,yes,no,telephone,may,fri,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,yes
4,38,unemployed,married,basic.4y,unknown,no,no,telephone,jun,thu,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
5,47,admin.,single,unknown,unknown,yes,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,33,admin.,married,university.degree,no,no,yes,telephone,jun,tue,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
7,31,technician,single,university.degree,no,yes,no,cellular,jul,mon,...,4,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
8,31,admin.,divorced,high.school,no,no,yes,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
9,51,housemaid,married,basic.9y,no,yes,no,cellular,jul,tue,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,no


今回は、この中で、marital（婚姻状況）を予測するモデルの構築を行ってもらいます。

### Step 2　使用する列の抽出
次に、使用する列だけを取り出します。　今回はmaritalと、予測に使えそうな age, housing を取り出します。

In [9]:
y_ = df_train_data.marital# ラベルデータ

In [10]:
DataFrame(y_).head(10)

,marital
0,married
1,married
2,divorced
3,married
4,married
5,single
6,married
7,single
8,divorced
9,married


ラベルデータの上位10件を表示しました。

In [11]:
X_ = df_train_data[['age', 'housing']]

In [12]:
X_.head(10)

,age,housing
0,58,yes
1,38,yes
2,81,yes
3,49,yes
4,38,no
5,47,yes
6,33,no
7,31,yes
8,31,no
9,51,yes


特徴量データの上位10件を表示しました。

### Step3　特徴量データの前処理
識別器での学習を行う前に、特徴量を適切な形へ変換する必要があります。
その過程を見ていきましょう。

まず、欠損値を含むデータの扱いですが、本課題では、簡単のため単純に削除する手法を取ります。

In [13]:
df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')# 欠損値を含むデータの削除
df_temp.index = range(len(df_temp)) 
y = df_temp.marital
X = df_temp[['age', 'housing']]

次に、housing 列は文字列であるため、数値に変換してあげる必要があります。ここでは、yes = 2, unknown = 1, no = 0 として変換します。

In [14]:
X.tail(10)

,age,housing
19990,33,yes
19991,41,yes
19992,54,no
19993,44,yes
19994,59,no
19995,24,no
19996,42,yes
19997,31,no
19998,25,yes
19999,33,no


In [29]:
housing_le = LabelEncoder()
X.housing = housing_le.fit_transform(X.housing)

In [30]:
X.tail(10)

,age,housing
19990,0.197531,2
19991,0.296296,2
19992,0.456790,0
19993,0.333333,2
19994,0.518519,0
19995,0.086420,0
19996,0.308642,2
19997,0.172840,0
19998,0.098765,2
19999,0.197531,0


これで、数値データに置き換えられました。


最後に、それぞれの特徴量の数値の変動量に差による影響力の差を排除するため、正規化を行います。

In [17]:
mm_scaler = MinMaxScaler()

In [18]:
X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))

In [19]:
X.tail(10)

,age,housing
19990,0.197531,1.0
19991,0.296296,1.0
19992,0.456790,0.0
19993,0.333333,1.0
19994,0.518519,0.0
19995,0.086420,0.0
19996,0.308642,1.0
19997,0.172840,0.0
19998,0.098765,1.0
19999,0.197531,0.0


これで、正規化が出来ました。

### Step 4　識別器の実装
ロジスティック回帰の例を参考に、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### ロジスティック回帰
ロジスティック回帰の例を示します。実装にはscikit-learn(http://scikit-learn.org/stable/) の関数を使用します。

In [20]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割

In [40]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得

print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

accuracy:0.653


上記の LogisticRegression( ) の部分を書き換えて、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### SVM
ここで、SVM (SVC) の実装を行ってください。特徴量も自由に選択して構いません。

In [1]:
svm = SVC(kernel="poly", degree=3, coef0=1, C=5)
svm.fit(X_train, y_train)

y_pred =svm.predict(X_test)

print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

NameError: name 'SVC' is not defined

#### K-近傍法

ここで、K-近傍法 (KNeighborsClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [23]:
knn =

y_pred =

print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

SyntaxError: invalid syntax (<ipython-input-23-b7f1e7a76ae0>, line 1)

#### 決定木

ここで、決定木 (DecisionTreeClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [ ]:
decition_tree = 

y_pred = 

print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

#### ランダムフォレスト

ここで、ランダムフォレスト (RandomForestClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [ ]:
random_forest = 

y_pred = 

print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

# 課題２: モデルの改善を行おう！
課題1で作成した各モデルについて、どうすればさらに良い精度を出すことができるのか、パラメータの調整や特徴量の選定などを行いましょう。
最後に、この課題において最善と考える特徴量とモデルを述べ、その理由も示してください。

In [24]:
from IPython.display import display
import pandas as pd
from pandas import DataFrame, Series
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

まず、選定した特徴量を用いて、課題１で行ったStep1〜Step3 を行いましょう。ただし、Step 2〜Step 3の処理は、preprocessingという関数で定義してください。ここで定義した関数を用いて評価を行います。
引数には、読み込んだデータを指定してください。返り値には、前処理を行った後の選定した特徴量 ( X )とmarital ( y )を指定してください。

In [25]:
### Step 1　　データの読み込み
df_train_data = pd.read_csv('train.csv')


### 課題１で行ったStep 2〜Step 3を関数として定義してください。
def preprocessing(df_train_data):
    ### Step 2　　使用する列の抽出
    y_ = df_train_data.marital# ラベルデータ
    X_ = df_train_data[['age', 'housing']]
    
    ### Step 3　　特徴量データの前処理
    # 欠損値を含むデータの削除
    df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')
    df_temp.index = range(len(df_temp)) 
    y = df_temp.marital
    X = df_temp[['age', 'housing']]
    
    # 文字列を数値データへ変換
    housing_le = LabelEncoder()
    X.housing = housing_le.fit_transform(X.housing)
    
    # 数値データの正規化
    mm_scaler = MinMaxScaler()
    X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
    X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))
    
    return X, y

上記で定義した関数を実行します。

In [26]:
X, y = preprocessing(df_train_data)

次に、この課題における最善なモデルを構築しましょう。課題１で使用したモデル以外を使用しても構いません。
下記は、ロジスティック回帰の場合の例です。

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割

In [28]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得
print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

accuracy:0.653


### 考察１
この課題において最善と考える特徴量を述べ、その理由も示してください。

（ここに記入してください）

### 考察２
この課題において最善と考えるモデルを述べ、その理由も示してください。

（ここに記入してください）

最後に、以下の best_model に構築したモデルを記入してください。この best_model を使用して精度評価を行います。

In [24]:
best_model = logistic_regression     # この例では、ロジスティック回帰を選択したので、logistic_regression を記入しています。
pickle.dump(best_model, open('best_model.pkl','wb'))

### --------評価用--------
ここから下は評価用となります。上の欄にモデルを記入した後は、配布資料に書いてある手順に沿ってファイルを提出してください。

In [ ]:
# df_test_data = pd.read_csv('bank_test.csv')
# preprocessing()
# X_test = X
# y_test = y

In [ ]:
# best2_model = pickle.load(open('best_model.pkl', 'rb'))
# y_pred =best2_model.predict(X_test)
# print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test)))